# Monitoring Mangrove Extents

## Background
[Global Mangrove Watch (GMW)](https://www.globalmangrovewatch.org/) is an initiative to track global mangrove extents. Using ALOS PALSAR and Landsat (optical) data to form a baseline extent of mangroves for the years 1996, 2007, 2008, 2009, 2010, 2015, and 2016 . More detailed information on the initiative can be found [here](http://data.unep-wcmc.org/datasets/45).


## Description
The goal of this notebook is to monitor the changing extent of mangroves, using the GMW layers as a baseline mask.

The process begins with retrieving Sentnel-2 data for a specific area in a time series. This dataset is then compressed into a median composite for each year. From the composite, we then calculate the NDVI values of each pixel in each year. The dataset is then masked, and the NDVI threshold is applied for mangrove classification. The following image shows the decision tree for the classification.

<img align="center" src="../resources/mangrove_decision_tree.png" width=600>

After classification, we can perform a variety of analyses on the data:
- We can estimate the change in mangrove areas by counting all the classified pixels for each year and plotting the trend line of the count. 
- We can also visualize the mangrove areas by plotting each classified pixel a certain color. 

## Getting started

**To run this analysis**, run all the cells in the notebook, starting with the [Load packages](#Load-packages) cell.

**After finishing the analysis**, return to the [Analysis parameters](#Analysis-parameters) cell, modify some values (e.g. choose a your own mangrove shapefile or time period to analyse) and re-run the analysis.

### Load packages

In [ ]:
%matplotlib inline

import warnings
import datacube
from datacube.utils import masking
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from requests import Request
import matplotlib.pyplot as plt
from datacube.utils import geometry
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap

from dea_tools.datahandling import mostcommon_crs
from dea_tools.plotting import display_map

import sys
try:
    from dea_tools.spatial import xr_rasterize
except ImportError:
    # Local copy of selected dea_tools
    if '../tools/' not in sys.path:
        sys.path.append('../tools/')
    from datacube_utils import xr_rasterize

### Set up a Dask cluster

Dask can be used to better manage memory use down and conduct the analysis in parallel. 

To use Dask, set up the local computing cluster using the cell below.

In [ ]:
from datacube.utils.dask import start_local_dask

client = start_local_dask()

display(client)

### Load the data

Connect to the datacube database and set up a processing cluster.

In [ ]:
dc = datacube.Datacube(app="Mangrove")

### Analysis parameters

* `lat, lon, buffer`: center lat/lon and analysis window size for the area of interest
* `product_name`: The name of the satellite product to use.
* `time_range`: The date range to analyse (e.g. `("2017", "2022")`).

**If running the notebook for the first time**, keep the default settings below.
This will demonstrate how the analysis works and provide meaningful results.


In [ ]:
top, left = 8.654161, 104.714182
bottom, right = 8.628812, 104.746726

lat_range = (bottom, top)
lon_range = (left, right)

product_name = "s2_l2a"

time_range = ("2017", "2022")

### View the selected location

The next cell will display the selected area on an interactive map. Feel free to zoom in and out to get a better understanding of the area you'll be analysing. Clicking on any point of the map will reveal the latitude and longitude coordinates of that point.

In [ ]:
display_map(lon_range, lat_range)

## Load cloud-masked Sentinel-2 data

The code below uses the `load_ard` function to load in data from the Sentinel-2 satellites for the area and time specified.
For more information, see the [Using load_ard notebook](../Frequently_used_code/Using_load_ard.ipynb).
The function will also automatically mask out clouds from the dataset, allowing us to focus on pixels that contain useful data:

In [ ]:
# General query
query = {
    "time": time_range,
    'x': lon_range,
    'y': lat_range,
}

# Identify the most common CRS
common_crs = mostcommon_crs(dc, product_name, query)
print(f"Selected CRS: {common_crs}")

# Update the query with additional parameters
query.update(
    {
        "product": product_name,
        "measurements": ["red", "nir_1", "mask"],
        "group_by": "solar_day",
        "resolution": (-10, 10),
        "output_crs": common_crs,
        "dask_chunks": {"time": 1, "x": -1, "y": -1}
    }
)

In [ ]:
ds_s2 = dc.load(**query)

ds_s2

### Masking

Clouds, cloud shadows, and other invalid pixels need to be removed before conducting the analysis.

The first step in masking is to identify the information in the masking band. This can be done using the `masking.describe_variable_flags)` function:

In [ ]:
masking.describe_variable_flags(ds_s2.mask)

For Sentinel-2 in the Asia EASI Hub, the masking information is stored in the **values** column of the **qa** row in this dataframe. To get a better look at the types of pixel classifications this mask has, we can display the values column:

In [ ]:
masking.describe_variable_flags(ds_s2.mask).loc["qa", "values"]

We can now construct a mask of valid pixel classes. In this case, we say that `vegetation`, `bare soils`, `water`, or `snow or ice` are valid, cloud-free pixels:

In [ ]:
# Multiple flags are combined as logial OR using the | symbol
cloud_free_mask = (
    masking.make_mask(ds_s2.mask, qa="vegetation") | 
    masking.make_mask(ds_s2.mask, qa="bare soils") |
    masking.make_mask(ds_s2.mask, qa="water") |
    masking.make_mask(ds_s2.mask, qa="snow or ice")
)

The final step is to mask the data.

In [ ]:
# Mask the data
ds_s2 = ds_s2.where(cloud_free_mask)

In [ ]:
# View the loaded data

ds_s2

## Load in the Global Mangrove Watch

Below we load the latest GMW layer, which is from 2016. We send a query to the [EASI Geoserver](https://geoserver.asia.easi-eo.solutions/), requesting the data using the `wfs_request_params` dictionary below.

In [ ]:
WFS_URL = "https://geoserver.asia.easi-eo.solutions/geoserver/ows?"
WFS_LAYER = "unep-wcmc:GMW_2016_v2"

wfs_request_params = {
    "service": "WFS",
    "version": "1.0.0",
    "request": "GetFeature",
    "typeName": WFS_LAYER,
    "outputFormat": "json",
    "bbox": f"{left},{bottom},{right},{top},EPSG:4326"
}

# Parse the URL with parameters
request_url = Request("GET", WFS_URL, params=wfs_request_params).prepare().url

# Read the data
gmw_2016_gdf = gpd.read_file(request_url)

# Examine first five rows of the data
gmw_2016_gdf.head()

In [ ]:
# Display loaded data on interactive map
gmw_2016_gdf.explore()

## Combine observations into noise-free summary images

In the code below, we take the time series of images and combine them into single images for each year using the `median NDVI`. The *Normalised Difference Vegetation Index* (NDVI) shows vegetation and is used for mangrove classification within the mangrove mask.

$$
\begin{aligned}
\text{NDVI} & = \frac{(\text{NIR} - \text{Red})}{(\text{NIR} + \text{Red})} \\
\end{aligned}
$$

For more information on indices, see the [Calculating Band Indices](../Frequently_used_code/Calculating_band_indices.ipynb) notebook.


> Note: This step can take several minutes to load if the study area is large. We recommend opening the Dask processing window to view the different computations that are being executed; to do this, see the Dask dashboard in DE Africa section of the Dask notebook.

In [ ]:
# Calculate MNDWI and add it to the loaded data set
ds_s2["NDVI"] = (ds_s2.nir_1 - ds_s2.red)/(ds_s2.nir_1 + ds_s2.red)

In [ ]:
# generate median annual summaries of NDVI
ds_summaries = ds_s2.NDVI.groupby("time.year").median().compute()

In [ ]:
# Plot the output summary images
ds_summaries.plot(col="year", cmap="YlGn", col_wrap=len(ds_summaries.year.values), vmin=0, vmax=1.0);

## Mangrove classification

### Apply a mask of the mangrove area

We will use the mangrove mask from the Global Mangrove Watch dataset so we can work with only the pixels in the mangrove area.  

In [ ]:
gmw_2016_raster = xr_rasterize(
    gdf=gmw_2016_gdf,
    da=ds_summaries,
    transform=ds_summaries.geobox.transform,
    crs=common_crs
)

In [ ]:
# Mask dataset to set pixels outside the GMW layer to `NaN`
ds_summaries_masked = ds_summaries.where(gmw_2016_raster)

In [ ]:
# Plot the output summary images
ds_summaries_masked.plot(col="year", cmap="YlGn", col_wrap=len(ds_summaries.year.values), vmin=0, vmax=1.0);

### Calculating regular and closed mangroves

Using the [mangrove decision tree](#Description) above, we can classify the pixels as:

* **mangroves:** pixels in the mangrove area with an `NDVI > 0.4`
    * **regular mangroves:** mangroves with `NDVI <= 0.7`
    * **closed mangroves:** mangroves with `NDVI > 0.7`

In [ ]:
all_mangroves = xr.where(ds_summaries_masked > 0.4, 1, np.nan)

regular_mangroves = all_mangroves.where(ds_summaries_masked <= 0.7)
closed_mangroves = all_mangroves.where(ds_summaries_masked > 0.7)

### Plot the mangrove types

Again we will combine the mangrove types for easier plotting.

In [ ]:
mangroves = xr.concat(
    [regular_mangroves, closed_mangroves, all_mangroves],
    dim=pd.Index(["regular", "closed", "total"], name="mangrove_type"),
)

regular_color = "gold"
closed_color = "green"

# Create a FacetGrid, so there is a subplot for each year
grid = xr.plot.FacetGrid(
    mangroves,
    col="year",
    col_wrap=len(ds_summaries.year.values),
    size=6,
    aspect=mangroves.x.size / mangroves.y.size,
)

# Define the sub-plot of mangrove types with a legend on a background of grey-scale NDVI
def plot_mangrove(data, ax, **kwargs):
    ds_summaries.sel(year=data.year).plot.imshow(
        ax=ax, cmap="Greys", vmin=-1, vmax=1, add_colorbar=False, add_labels=False
    )
    data.sel(mangrove_type="regular").plot.imshow(
        ax=ax,
        cmap=ListedColormap([regular_color]),
        add_colorbar=False,
        add_labels=False,
    )
    data.sel(mangrove_type="closed").plot.imshow(
        ax=ax, cmap=ListedColormap([closed_color]), add_colorbar=False, add_labels=False
    )

    ax.legend(
        [Patch(facecolor=regular_color), Patch(facecolor=closed_color)],
        ["Regular", "Closed"],
        loc="lower right",
    )

# Plot the each year sub-plot
grid.map_dataarray(plot_mangrove, x="x", y="y", add_colorbar=False)

# Update sub-plot titles
for i, name in np.ndenumerate(grid.name_dicts):
    grid.axes[i].title.set_text(str(name["year"]));

## Mangrove change
### Plot the change of mangrove classification in area over time

In [ ]:
# Convert pixel count to km^2
m2_per_ha = 10000
m2_per_pixel = query["resolution"][1] ** 2
mangrove_area = mangroves.sum(dim=("x", "y")) * m2_per_pixel / m2_per_ha

# Fix axis and legend text
mangrove_area.name = r"Area (Ha)"
mangrove_area = mangrove_area.rename(mangrove_type="Mangrove type")

# Plot the line graph
mangrove_area.plot(
    hue="Mangrove type",
    x="year",
    xticks=mangrove_area.year,
    size=6,
    linestyle="--",
    marker="o",
)
plt.title("Mangrove classification over time");

### Monitoring change in mangroves

We can calculate the growth and loss of mangroves between the first and last year in our timeseries

In [ ]:
total_mangroves = (mangroves.loc["total"] == 1).astype(int)

# Calculate the change in mangrove extent
old = total_mangroves.isel(year=0)
new = total_mangroves.isel(year=-1)
change = new - old

# reclassify into growth, loss and stable
growth = xr.where(change == 1, 1, np.nan)
loss = xr.where(change == -1, -1, np.nan)
stable = old.where(~change)
stable = xr.where(stable == 1, 1, np.nan)

#### Plot the change

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

ds_summaries.isel(year=0).plot.imshow(
    ax=ax, cmap="Greys", vmin=-1, vmax=1, add_colorbar=False, add_labels=False
)
stable.plot(
    ax=ax, cmap=ListedColormap(["palegoldenrod"]), add_colorbar=False, add_labels=False
)
growth.squeeze().plot.imshow(
    ax=ax, cmap=ListedColormap(["lime"]), add_colorbar=False, add_labels=False
)
loss.plot.imshow(
    ax=ax, cmap=ListedColormap(["fuchsia"]), add_colorbar=False, add_labels=False
)
ax.legend(
    [
        Patch(facecolor='lime'),
        Patch(facecolor='fuchsia'),
        Patch(facecolor="palegoldenrod"),
    ],
    ["New mangroves", "Loss of mangroves", "Stable mangroves"],
    loc="lower right",
)
plt.title('Change in mangrove extent between {} and {}'.format(ds_summaries.year.values[0], ds_summaries.year.values[-1]));

### Determine the net change between years

In [ ]:
loss_count = loss.sum().values * m2_per_pixel / m2_per_ha
gain_count = growth.sum().values * m2_per_pixel / m2_per_ha
stable_count = stable.sum().values* m2_per_pixel / m2_per_ha

counts = {
          'Loss of Mangroves': loss_count,
          'New Mangroves': gain_count,
          'No Change': stable_count,
          'Delta Change': gain_count + loss_count
          }

df = pd.DataFrame(counts, index=['Area (Ha)'])
df.plot.barh(color=['tab:red','tab:green','tab:orange','tab:blue'])
plt.title('Change in mangrove extent between {} and {}'.format(ds_summaries.year.values[0], ds_summaries.year.values[-1]));


## Advanced: Compare Recent Mangrove Extent to GMW 1996

In the previous plots, we were calculating the change in mangrove extent from the start of the Sentinel-2 time series (2017) until now (2021). However, the baseline mangrove extent map, the Global Mangrove Watch (GMW) layer goes all the way back to 1996. Thus, the mangrove extent may have change significantly between 1996 and the beginning of the Sentinel-2 archive.  We can use the GMW layer to inspect how mangrove extent has changed since the development of the 2016 product that we examined above. 

A word of caution here, the GMW product was developed using a different methodology to the one used here, so differences in mangrove extent may be attributable to the different methods. Nevertheless, this should still provide us with a reasonable indication of the changes that have occurred between 1996 and the beginning of the Sentinel-2 archive in 2017.

### Load the earliest GMW layer

In [ ]:
WFS_URL = "https://geoserver.asia.easi-eo.solutions/geoserver/ows?"
WFS_LAYER = "unep-wcmc:GMW_1996_v2"

wfs_request_params = {
    "service": "WFS",
    "version": "1.0.0",
    "request": "GetFeature",
    "typeName": WFS_LAYER,
    "outputFormat": "json",
    "bbox": f"{left},{bottom},{right},{top},EPSG:4326"
}

# Parse the URL with parameters
request_url = Request("GET", WFS_URL, params=wfs_request_params).prepare().url

# Read the data
gmw_1996_gdf = gpd.read_file(request_url)

# Examine first five rows of the data
gmw_1996_gdf.head()

In [ ]:
gmw_1996_raster = xr_rasterize(
    gdf=gmw_1996_gdf,
    da=ds_summaries,
    transform=ds_summaries.geobox.transform,
    crs=common_crs
)

### Determine change between 1996 and 2017

In [ ]:
#Find the change between now and the baseline
baseline_change = old - gmw_1996_raster

# reclassify into growth, loss and stable
growth_bs = xr.where(baseline_change == 1, 1, np.nan)
loss_bs = xr.where(baseline_change == -1, -1, np.nan)
stable_bs = gmw_1996_raster.where(~baseline_change)
stable_bs = xr.where(stable == 1, 1, np.nan)

#### Plot the change

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

ds_summaries.isel(year=0).plot.imshow(
    ax=ax, cmap="Greys", vmin=-1, vmax=1, add_colorbar=False, add_labels=False
)
stable_bs.plot(
    ax=ax, cmap=ListedColormap(["palegoldenrod"]), add_colorbar=False, add_labels=False
)
growth_bs.plot.imshow(
    ax=ax, cmap=ListedColormap(["lime"]), add_colorbar=False, add_labels=False
)
loss_bs.plot.imshow(
    ax=ax, cmap=ListedColormap(["fuchsia"]), add_colorbar=False, add_labels=False
)
ax.legend(
    [
        Patch(facecolor='lime'),
        Patch(facecolor='fuchsia'),
        Patch(facecolor="palegoldenrod"),
    ],
    ["New mangroves", "Loss of mangroves", "Stable Mangroves"],
    loc="lower right",
)
plt.title('Change in mangrove extent between GMV 1996 baseline and {}'.format(ds_summaries.year.values[0]));

### Determine the net change between 1996 and 2017

In [ ]:
loss_bs_count = loss_bs.sum().values * m2_per_pixel / m2_per_ha
gain_bs_count = growth_bs.sum().values * m2_per_pixel / m2_per_ha
stable_bs_count = stable_bs.sum().values* m2_per_pixel / m2_per_ha

counts = {
          'Loss of Mangroves': loss_bs_count,
          'New Mangroves': gain_bs_count,
          'No Change': stable_bs_count,
          'Delta Change': gain_bs_count + loss_bs_count
          }

df = pd.DataFrame(counts, index=['Area (Ha)'])
df.plot.barh(color=['tab:red','tab:green','tab:orange','tab:blue'])
plt.title('Change in mangrove extent between GMV 2010 baseline and {}'.format(ds_summaries.year.values[0]));

## Drawing conclusions
Here are some questions to think about:

* What are the causes of mangrove growth and loss? 